In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import scipy as sp

In [ ]:
from matplotlib import pyplot as plt
import skimage
from skimage import (morphology, segmentation, exposure, feature, filters,
                     measure, transform, util, io, color)

In [ ]:
from toolz.curried import *

In [ ]:
# this will raise a warning about matplotlib backend

from spotz import imgz, gridder2, segmenter2, spotzplot

# User settings

In [ ]:
# change name of file
PLATE_IMG = "XXXX"

# change number of rows and cols as appropriate
NROWS = 12
NCOLS = 8

# change if you'd like larger or smaller figs
FIG_SIZE = (8, 10)

# Original image

In [ ]:
img = imgz.read_image(PLATE_IMG)

In [ ]:
fig = plt.figure(figsize = FIG_SIZE)
imgz.imshowg(img)
pass

# Invert image

In [ ]:
iimg = imgz.invert(img)

In [ ]:
fig = plt.figure(figsize = FIG_SIZE)
imgz.imshowg(iimg)
pass

# Gridding

## Thresholding and binary opening

In [ ]:
# initial thresholding and rotation correction
rbimg = pipe(iimg, 
             imgz.threshold_otsu,
             imgz.disk_opening(3), 
             imgz.clear_border, 
             gridder2.fix_rotation)


In [ ]:
fig = plt.figure(figsize = FIG_SIZE)
imgz.imshowg(rbimg)
pass

## Estimate grid coordinates and draw grid

In [ ]:
g = gridder2.find_grid(NROWS, NCOLS, rbimg)

In [ ]:
fig = plt.figure(figsize = FIG_SIZE)
imgz.imshowg(rbimg)
ax = plt.gca()
ax.plot(g.centers[:,1], g.centers[:,0], 'r.')
spotzplot.draw_bboxes(g.bboxes, ax)
pass

# Segmentation

## Rethreshold based on grid

In [ ]:
timg = pipe(iimg,
            segmenter2.threshold_bboxes(g.bboxes, threshold_func = imgz.threshold_li),
            imgz.remove_small_objects(50),
            imgz.remove_small_holes(25),
            imgz.disk_closing(3),
            imgz.clear_border)

In [ ]:
fig = plt.figure(figsize = FIG_SIZE)
imgz.imshowg(timg)
pass

## Segmentation by watershed

### Filter objects by grid

In [ ]:
filtered_img, filtered_regions = segmenter2.assign_objects_to_grid(g.centers, 
                                                                   timg, 
                                                                   maxdist = 30) 
filtered_bboxes = [r.bbox if r else None for r in filtered_regions]



In [ ]:
fig = plt.figure(figsize = FIG_SIZE)
imgz.imshowg(filtered_img > 0)
pass

In [ ]:
wshed = segmenter2.watershed_segment_bboxes(g.centers, 
                                             filtered_bboxes, 
                                             iimg, timg, 
                                             seed_width = 5)

In [ ]:
fig = plt.figure(figsize = FIG_SIZE)
ax = plt.gca()
ax.imshow(color.label2rgb(wshed, img, bg_label = 0))
pass

In [ ]:
fig, ax = spotzplot.draw_image_and_labels(img, wshed,
                                          mask_cmap = "Reds", alpha = 0.35,
                                         fontsize = 10, textcolor = "orange")
fig.set_size_inches(FIG_SIZE)